In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Cargar el módulo de Mediapipe para detección de manos
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Definir la carpeta de videos
videos_folder = "CUANDO_AUG"

# Definir la carpeta de salida de las imágenes resumen
output_folder = "CUANDO_IMG_AUG"

# Asegurarse de que la carpeta de salida exista
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Obtener la lista de archivos de video en la carpeta
video_files = [f for f in os.listdir(videos_folder) if os.path.isfile(os.path.join(videos_folder, f)) and f.endswith((".mp4", ".avi"))]

video_folder_name = os.path.basename(videos_folder)
contador=161

# Configurar Mediapipe para la detección de manos
with mp_hands.Hands(static_image_mode=True, max_num_hands=2,
                    min_detection_confidence=0.05, min_tracking_confidence=0.05) as hands:

    for video_file in video_files:
        video_path = os.path.join(videos_folder, video_file)
        cap = cv2.VideoCapture(video_path)

        frame_count = 0
        summary_image = np.zeros((396, 704, 3))

        while cap.isOpened():
            # Leer el siguiente frame
            success, image = cap.read()
            black_image = np.zeros_like(image)
            colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0)] 
            desired_keypoints = [5, 9, 13] 
            if not success:
                break

            # Detección de manos
            results_hands = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if results_hands.multi_hand_landmarks:
                for hand_landmarks in results_hands.multi_hand_landmarks:
                    for idx, landmark in enumerate(hand_landmarks.landmark):
                        if idx in desired_keypoints:
                            x = int(landmark.x * image.shape[1])
                            y = int(landmark.y * image.shape[0])

                            color = colors[desired_keypoints.index(idx)]  # Obtener el color correspondiente al punto clave
                            cv2.circle(black_image, (x, y), 4, color, -1)

                summary_image += black_image.astype(np.float32)

            frame_count += 1   
            if frame_count == 100:
                break

        summary_image = np.clip(summary_image, 0, 255).astype(np.uint8)

        #obtener nombre de la carpeta
        #carpeta = video_file.split("_")[0]
        image_number = str(contador).zfill(3)
        # Obtener el nombre base del archivo de video
        #video_base_name = os.path.splitext(video_file)[0]

        # Generar el nombre de salida de la imagen resumen
        #summary_filename = f"{video_folder_name}{image_number}.jpg"
        summary_output_name = f"{video_folder_name}{image_number}.jpg"
        summary_output_path = os.path.join(output_folder, summary_output_name)

        # Guardar la imagen resumen
        cv2.imwrite(summary_output_path, summary_image)
        contador+=1

        # Liberar recursos
        cap.release()

# Cerrar las ventanas abiertas por OpenCV
cv2.destroyAllWindows()